In [1]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch


mirrored_strategy = tf.distribute.MirroredStrategy()
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

2024-04-21 22:47:14.690751: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 22:47:15.267156: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/undergrad/miniconda3/envs/tf-2.12.0/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
input_text = '아이들의 정서적인 발달에 동화책은'
input_ids = tokenizer.encode(input_text, return_tensors='tf')
#term = 3

last_index = input_ids.shape[-1]

with mirrored_strategy.scope():
    while input_ids.shape[-1] < 100:
        print("================================")
        print(input_ids.shape[-1])
        #set 나눠서 green set 로짓을 촉진하기
        gamma = 1.0
        delta = 0.0
        Logits = model(input_ids).logits
        print(f"original Logits : {Logits}")
        
        #seed 생성
        seed = input_ids[-1] % Logits.shape[-1]
        print(f"seed : {seed[-1]}")
        np.random.seed(seed[-1])

        #random green list 설정
        indices = np.arange(Logits.shape[-1]) #0~51999
        green_list_size = int(Logits.shape[-1] * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
        print(f"green list : {green_list} {green_list.shape}") #25600개의 green list

        #--여기부터 probability 구하기
        # green logit과 red logit의 값을 담을 리스트 생성
        green_logits = tf.zeros(green_list_size)
        red_logits = tf.zeros(Logits.shape[-1] - green_list_size)

        #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
        Logits_num = np.zeros((0,last_index-1,Logits.shape[-1]))
        Logits_num = Logits.numpy()
        for i in green_list :
            Logits_num[0,last_index-1,i] = Logits_num[0,last_index-1,i]+delta
        Logits = tf.convert_to_tensor(Logits_num, dtype=tf.float32)
        print(f"updated Logits : {Logits}")
        
        # softmax 계산을 위한 분모
        denominator = tf.reduce_sum(tf.exp(Logits), axis=-1)  
        print(f"denominator : {denominator}") #25600개의 green list

        # Logits 텐서에 softmax 적용하여 확률 계산 (병렬 계산)
        softmax_tensor = Logits.numpy()
        softmax_tensor[0, last_index-1, :] = np.exp(softmax_tensor[0, last_index-1, :])
        softmax_tensor[0, last_index-1, :] /= denominator[0,last_index-1]
        print(softmax_tensor.shape)
        print(f"Probabilities : {softmax_tensor}")

        # 데이터 타입 확인 및 조정
        add_token_id = tf.argmax(softmax_tensor[0][last_index-1])
        print(f"updated Logits Max value : {add_token_id}")


        # 텐서 연결
        add_token_id = tf.reshape(add_token_id, [1,1])
        add_token_id = tf.cast(add_token_id, input_ids.dtype)
        input_ids = tf.concat([input_ids, add_token_id], axis=-1)
        next_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(f"next token : {next_token}")
        
        last_index = input_ids.shape[-1]
        #term = 3

NameError: name 'tokenizer' is not defined